In [1]:
import pandas as pd 

data = pd.read_csv('final_movies_with_birthplaces.csv')
data.head()

,movie,director,m_year,critic,crit_org,crit_cn,nm_dir,birthplace,cleaned_birthplace
0,Mulholland Drive,David Lynch,2001,Simon Abrams,Freelance film critic,US,1,"Missoula, Montana, United States","Missoula, Montana, United States"
1,In the Mood for Love,Wong Kar-wai,2000,Simon Abrams,Freelance film critic,US,1,"Shanghai, China","Shanghai, China"
2,The Tree of Life,Terrence Malick,2011,Simon Abrams,Freelance film critic,US,1,"Ottawa, Illinois, United States","Ottawa, Illinois, United States"
3,Yi Yi: A One and a Two,Edward Yang,2000,Simon Abrams,Freelance film critic,US,1,"Shanghai, Republic of China","Shanghai, Republic of China"
4,Goodbye to Language,Jean-Luc Godard,2014,Simon Abrams,Freelance film critic,US,1,"Paris, France","Paris, France"


In [2]:
# Extract unique birthplaces for geocoding
unique_birthplaces = data['birthplace'].dropna().unique()

# Create a DataFrame to manage geocoding results
geocode_df = pd.DataFrame(unique_birthplaces, columns=['birthplace'])
geocode_df.head()


,birthplace
0,"Missoula, Montana, United States"
1,"Shanghai, China"
2,"Ottawa, Illinois, United States"
3,"Shanghai, Republic of China"
4,"Paris, France"


In [3]:
# from geopy.geocoders import Nominatim
# from concurrent.futures import ThreadPoolExecutor

# geolocator = Nominatim(user_agent="geojson_mapping")

# # Function to geocode a single location
# def geocode_location(location):
#     try:
#         loc = geolocator.geocode(location, timeout=10)
#         if loc:
#             return loc.latitude, loc.longitude
#     except:
#         pass
#     return None, None

# # Parallelize the geocoding
# with ThreadPoolExecutor() as executor:
#     results = list(executor.map(geocode_location, df['birthplace']))

# df['latitude'], df['longitude'] = zip(*results)

# # Save the results
# df.to_csv('final_movies_with_coordinates.csv_1', index=False)
# print("Geocoding completed using parallel processing.")


In [4]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

# Load the CSV file
file_path = 'BFI.csv'
df = pd.read_csv(file_path)

# Initialize the geolocator
geolocator = Nominatim(user_agent="geojson_mapping")
# Function to get coordinates
def get_coordinates(location):
    try:
        location_data = geolocator.geocode(location, timeout=10)
        if location_data:
            return location_data.latitude, location_data.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        return None, None

# Add columns for latitude and longitude
df['latitude'], df['longitude'] = zip(*df['Country'].apply(get_coordinates))

# Save the updated DataFrame with coordinates
output_path = 'BFI_final_movies_with_coordinates.csv'
df.to_csv(output_path, index=False)

print(f"Updated file saved at {output_path}")


Updated file saved at BFI_final_movies_with_coordinates.csv


In [5]:
data = pd.read_csv('final_movies_with_coordinates.csv')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'final_movies_with_coordinates.csv'

In [6]:
import json
json_data = json.loads(df.to_json(orient='records'))

# Save or print the JSON object
output_path = 'BFI_final_movies_with_coordinates.json'
with open(output_path, 'w') as json_file:
    json.dump(json_data, json_file, indent=2)

print(f"JSON file saved to {output_path}")

JSON file saved to BFI_final_movies_with_coordinates.json


In [14]:
import pandas as pd
import json

# Load the CSV file
csv_path = "BFI_final_movies_with_coordinates.csv"  # Update with the correct file path
bfi_data = pd.read_csv(csv_path)

# Inspect the data (optional)
print(bfi_data.head())

# Define a function to create GeoJSON features
def create_geojson_feature(row):
    return {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row['longitude'], row['latitude']]
        },
        "properties": {
            "headline": f"{row['Director']} ({row['Country']})",
            "article": row['Films'],
            "group_name": row['Director']
        }
    }

# Generate GeoJSON structure
features = bfi_data.apply(create_geojson_feature, axis=1).tolist()
geojson_data = {
    "type": "FeatureCollection",
    "features": features
}

# Save the GeoJSON as a JavaScript variable
js_path = "bfi-geo-data.js"  # Update with the desired output path
with open(js_path, "w") as js_file:
    js_file.write(f"const infoData = {json.dumps(geojson_data, indent=2)};")

print(f"Updated JavaScript file saved to {js_path}")


                Films                     Year  Year1              Country  \
0      Born in Flames                     1983   1983                  NaN   
1       Pandora's Box              1928Germany   1928              Germany   
2  Sullivan's Travels                  1941USA   1941                  USA   
3          Annie Hall                  1977USA   1977                  USA   
4               Earth  1930USSR, Ukrainian SSR   1930  USSR, Ukrainian SSR   

                         Director   latitude   longitude  
0        Directed byLizzie Borden  46.314475   11.048029  
1           Directed byG.W. Pabst  51.163818   10.447831  
2      Directed byPreston Sturges  39.783730 -100.445882  
3          Directed byWoody Allen  39.783730 -100.445882  
4  Directed byAlexander Dovzhenko        NaN         NaN  
Updated JavaScript file saved to bfi-geo-data.js


In [9]:
print(bfi_data.columns)


Index(['Films', 'Year', 'Year1', 'Country', 'Director', 'latitude',
       'longitude'],
      dtype='object')


In [ ]:
json_data = json.loads(df.to_json(orient='records'))

# Save or print the JSON object
output_path = 'final_movies_with_coordinates.json'
with open(output_path, 'w') as json_file:
    json.dump(json_data, json_file, indent=2)

print(f"JSON file saved to {output_path}")

In [31]:
# Suppress printing the entire GeoJSON structure
import json

# Load GeoJSON file (large file example)
geojson_file = 'custom.geo.json'
with open(geojson_file, 'r') as f:
    geo_format = json.load(f)

# Print only a summary or specific parts
print("Number of features:", len(geo_format['features']))
print("First feature properties:", geo_format['features'][0]['properties'])


Number of features: 258
First feature properties: {'featurecla': 'Admin-0 country', 'scalerank': 0, 'labelrank': 5, 'sovereignt': 'Costa Rica', 'sov_a3': 'CRI', 'adm0_dif': 0, 'level': 2, 'type': 'Sovereign country', 'tlc': '1', 'admin': 'Costa Rica', 'adm0_a3': 'CRI', 'geou_dif': 0, 'geounit': 'Costa Rica', 'gu_a3': 'CRI', 'su_dif': 0, 'subunit': 'Costa Rica', 'su_a3': 'CRI', 'brk_diff': 0, 'name': 'Costa Rica', 'name_long': 'Costa Rica', 'brk_a3': 'CRI', 'brk_name': 'Costa Rica', 'brk_group': None, 'abbrev': 'C.R.', 'postal': 'CR', 'formal_en': 'Republic of Costa Rica', 'formal_fr': None, 'name_ciawf': 'Costa Rica', 'note_adm0': None, 'note_brk': None, 'name_sort': 'Costa Rica', 'name_alt': None, 'mapcolor7': 3, 'mapcolor8': 2, 'mapcolor9': 4, 'mapcolor13': 2, 'pop_est': 5047561, 'pop_rank': 13, 'pop_year': 2019, 'gdp_md': 61801, 'gdp_year': 2019, 'economy': '5. Emerging region: G20', 'income_grp': '3. Upper middle income', 'fips_10': 'CS', 'iso_a2': 'CR', 'iso_a2_eh': 'CR', 'iso_a3'

In [32]:
import pandas as pd
import json

# Load the CSV file
file_path = 'final_movies_with_coordinates.csv'
df = pd.read_csv(file_path)

# Convert the DataFrame to GeoJSON format
def convert_to_geojson(dataframe):
    geojson = {"type": "FeatureCollection", "features": []}
    for _, row in dataframe.iterrows():
        feature = {
            "type": "Feature",
            "properties": {
                "birthplace": row['birthplace'],
                "movie": row['movie']
            },
            "geometry": {
                "type": "Point",
                "coordinates": [row['longitude'], row['latitude']]  # GeoJSON expects [lng, lat]
            }
        }
        geojson["features"].append(feature)
    return geojson

# Generate GeoJSON
geo_format = convert_to_geojson(df)

# Save GeoJSON to a JavaScript file
js_output_file = 'geo-dataAGAIN.js'
with open(js_output_file, 'w') as outfile:
    outfile.write("const infoData = ")
    json.dump(geo_format, outfile, indent=2)

print(f"JavaScript file with GeoJSON saved: {js_output_file}")


JavaScript file with GeoJSON saved: geo-dataAGAIN.js


In [20]:
import pandas as pd
import json

# Load the CSV with coordinates
file_path = 'final_movies_with_coordinates.csv'
df = pd.read_csv(file_path)

# Function to convert to GeoJSON
def create_geojson(df):
    geo_data = {"type": "FeatureCollection", "features": []}
    
    for _, row in df.iterrows():
        feature = {
            "type": "Feature",
            "properties": {
                "name": row['birthplace'],
                "movies": row['movie']  # Assuming 'movie' is a column
            },
            "geometry": {
                "type": "Point",
                "coordinates": [row['longitude'], row['latitude']]  # Ensure order is [lng, lat]
            }
        }
        geo_data['features'].append(feature)
    
    return geo_data

# Convert the DataFrame to GeoJSON
geo_format = create_geojson(df)

# Save GeoJSON to a file
output_geojson = 'geo-data_final.json'
with open(output_geojson, 'w') as geojson_file:
    json.dump(geo_format, geojson_file, indent=2)

print(f"GeoJSON file saved to {output_geojson}")


GeoJSON file saved to geo-data_final.json


In [21]:
# Save the GeoJSON as a JavaScript variable
js_output = 'geo-dataAGAIN.js'
with open(js_output, 'w') as outfile:
    outfile.write("const infoData = ")
    json.dump(geo_format, outfile)

print(f"JavaScript file created at {js_output}")


JavaScript file created at geo-dataAGAIN.js
